In [2]:
from dlgo.data.processor import GoDataProcessor
from dlgo.encoders.oneplane import OnePlaneEncoder

from dlgo.networks import small
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint  # <1>
# <1> With model checkpoints we can store progress for time-consuming experiments

In [7]:
if __name__ == '__main__':
    go_board_rows, go_board_cols = 19, 19
    num_classes = go_board_rows * go_board_cols
    num_games = 10000

    encoder = OnePlaneEncoder((go_board_rows, go_board_cols))  # <1>

    processor = GoDataProcessor(encoder=encoder.name())  # <2>

    X, Y = processor.load_go_data('train', num_games)  # <3>
    print('X.shape', X.shape)
    print('Y.shape', Y.shape)

    validationX, validationY = processor.load_go_data('test', num_games)
    print('validationX.shape', validationX.shape)
    print('validationY.shape', validationY.shape)
    # <1> First we create an encoder of board size.
    # <2> Then we initialize a Go Data processor with it.
    # <3> From the processor we create two data generators, for training and testing.


>>> Reading cached index page
KGS-2019_04-19-1255-.tar.gz 1255
KGS-2019_03-19-1478-.tar.gz 1478
KGS-2019_02-19-1412-.tar.gz 1412
KGS-2019_01-19-2095-.tar.gz 2095
KGS-2018_12-19-1992-.tar.gz 1992
KGS-2018_11-19-1879-.tar.gz 1879
KGS-2018_10-19-1209-.tar.gz 1209
KGS-2018_09-19-1587-.tar.gz 1587
KGS-2018_08-19-1447-.tar.gz 1447
KGS-2018_07-19-949-.tar.gz 949
KGS-2018_06-19-1002-.tar.gz 1002
KGS-2018_05-19-1590-.tar.gz 1590
KGS-2018_04-19-1612-.tar.gz 1612
KGS-2018_03-19-833-.tar.gz 833
KGS-2018_02-19-1167-.tar.gz 1167
KGS-2018_01-19-1526-.tar.gz 1526
KGS-2017_12-19-1488-.tar.gz 1488
KGS-2017_11-19-945-.tar.gz 945
KGS-2017_10-19-1351-.tar.gz 1351
KGS-2017_09-19-1353-.tar.gz 1353
KGS-2017_08-19-2205-.tar.gz 2205
KGS-2017_07-19-1191-.tar.gz 1191
KGS-2017_06-19-910-.tar.gz 910
KGS-2017_05-19-847-.tar.gz 847
KGS-2017_04-19-913-.tar.gz 913
KGS-2017_03-19-717-.tar.gz 717
KGS-2017_02-19-525-.tar.gz 525
KGS-2017_01-19-733-.tar.gz 733
KGS-2016_12-19-1208-.tar.gz 1208
KGS-2016_11-19-980-.tar.gz 980


In [8]:
    input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
    network_layers = small.layers(input_shape)
    model = Sequential()
    for layer in network_layers:
        model.add(layer)
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    print(model.summary())

    epochs = 5
    batch_size = 128
    model.fit(X, Y, batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(validationX, validationY),
                        callbacks=[ModelCheckpoint('../checkpoints/small_model_epoch_{epoch}.keras')])  # <5>
    # <5> After each epoch we persist a checkpoint of the model.
    model.evaluate(validationX, validationY, batch_size=batch_size)  # <6>
    # <6> For evaluation we also specify the validation set.

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_8                │ (None, 1, 25, 25)      │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 48, 19, 19)     │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 48, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_9                │ (None, 48, 23, 23)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 19, 19)     │        38,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 32, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_10               │ (None, 32, 23, 23)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 19, 19)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 32, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_11               │ (None, 32, 23, 23)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 19, 19)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 32, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 11552)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     5,915,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 361)            │       185,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,192,425 (23.62 MB)

 Trainable params: 6,192,425 (23.62 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
15328/15328 ━━━━━━━━━━━━━━━━━━━━ 914s 60ms/step - accuracy: 0.0068 - loss: 5.7087 - val_accuracy: 0.0363 - val_loss: 4.9895
Epoch 2/5
15328/15328 ━━━━━━━━━━━━━━━━━━━━ 812s 53ms/step - accuracy: 0.0513 - loss: 4.7588 - val_accuracy: 0.1343 - val_loss: 4.1741
Epoch 3/5
15328/15328 ━━━━━━━━━━━━━━━━━━━━ 818s 53ms/step - accuracy: 0.1720 - loss: 3.8599 - val_accuracy: 0.2292 - val_loss: 3.4326
Epoch 4/5
15328/15328 ━━━━━━━━━━━━━━━━━━━━ 823s 54ms/step - accuracy: 0.2562 - loss: 3.2346 - val_accuracy: 0.2436 - val_loss: 3.2776
Epoch 5/5
15328/15328 ━━━━━━━━━━━━━━━━━━━━ 823s 54ms/step - accuracy: 0.2836 - loss: 3.0364 - val_accuracy: 0.2476 - val_loss: 3.2158
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2643 - loss: 3.1349


[3.215824842453003, 0.24755859375]